# Trabalho Final Visão Computacional
# Eduardo Fontes e Guilherme Dórea

# Rastreamento de Pose de um Objeto com OpenCV

Este notebook tem como objetivo implementar um sistema para determinar a pose (posição e orientação) de uma caixa vermelha posicionada em cima de um robô móvel. O sistema deve rastrear a caixa em um vídeo, armazenar a pose para cada frame e gerar um novo vídeo com a trajetória plotada.

## 1. Importação das Bibliotecas

Primeiro, importamos as bibliotecas necessárias para o processamento de imagens e visualização.

In [45]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

## 2. Função para Interpolar a Curva de Movimento

A função `interpolate_curve` é responsável por plotar a curva interpolada do movimento do objeto rastreado.

In [46]:
def interpolate_curve(path):
    path = np.array(path)
    plt.figure()
    plt.plot(path[:, 0], path[:, 1], 'b-', label='Interpolated Path')
    plt.title("Interpolated Curve of Movement")
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.legend()
    plt.show()

## 3. Função Principal de Rastreamento

A função `track_husky` é a principal função do sistema. Ela realiza o rastreamento do objeto vermelho em um vídeo, aplica máscaras para filtrar regiões indesejadas, detecta o objeto e plota a trajetória.

In [47]:
def track_husky(video_path, name, outpath, display=True):
    cap = cv.VideoCapture(video_path)

    # Motion detection and background subtraction
    background_remover = cv.createBackgroundSubtractorKNN(history=500, dist2Threshold=400, detectShadows=True)

    frames = []
    path = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        result = frame.copy()

        # Convert to HSV for color filtering
        image = cv.cvtColor(frame, cv.COLOR_BGR2HSV)

        # Color filter for red
        lower_red = np.array([165, 50, 0])
        upper_red = np.array([180, 255, 255])
        color_mask = cv.inRange(image, lower_red, upper_red)
        red_isolated = cv.bitwise_and(result, result, mask=color_mask)

        # Define a rectangular region to exclude areas
        rect_start = (0, 450)
        rect_end = (600, 600)
        rec_mask = np.zeros(image.shape[:2], dtype=np.uint8)
        cv.rectangle(rec_mask, rect_start, rect_end, 255, thickness=cv.FILLED)
        inverse_rec_mask = cv.bitwise_not(rec_mask)

        # Define a triangular region to exclude areas
        triangle_mask = np.zeros(image.shape[:2], dtype=np.uint8)
        triangle_vertices = np.array([(350, 600), (1000, 600), (1000, 270)])
        cv.fillPoly(triangle_mask, [triangle_vertices], 255)
        inverse_triangle_mask = cv.bitwise_not(triangle_mask)

        # Combine masks
        combined_mask = cv.bitwise_and(inverse_rec_mask, inverse_triangle_mask)
        combined_masked = cv.bitwise_and(result, result, mask=combined_mask)

        # Apply Gaussian blur and background subtraction
        blurred = cv.GaussianBlur(combined_masked, (5, 5), 0)
        mask = background_remover.apply(color_mask)
        _, mask = cv.threshold(mask, 254, 255, cv.THRESH_BINARY)
        mask = cv.morphologyEx(mask, cv.MORPH_OPEN, np.ones((3, 3), np.uint8))
        mask = cv.bitwise_and(mask, combined_mask)

        # Find contours
        contours, _ = cv.findContours(mask, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)
        max_cnt = None
        max_area = 0

        for cnt in contours:
            area = cv.contourArea(cnt)
            if area > 800 and len(cnt) >= 5 and area > max_area:
                max_area = area
                max_cnt = cnt

        if max_cnt is not None:
            ellipse = cv.fitEllipse(max_cnt)
            M = cv.moments(max_cnt)
            centroid_x = int(M['m10'] / M['m00'])
            centroid_y = int(M['m01'] / M['m00'])

            # Track centroid
            path.append([centroid_x, centroid_y])

            cv.ellipse(frame, ellipse, (0, 255, 0), 2)

        # Draw path
        for point in path:
            cv.circle(frame, (point[0], point[1]), 2, (255, 255, 255), -1)

        # Combine visualizations
        top_row = np.hstack((red_isolated, combined_masked))
        bottom_row = np.hstack((blurred, frame))
        combined_image = np.vstack((top_row, bottom_row))

        # Resize the combined image to fit the screen
        scale_percent = 60  # Adjust this value to fit your screen
        width = int(combined_image.shape[1] * scale_percent / 100)
        height = int(combined_image.shape[0] * scale_percent / 100)
        dim = (width, height)
        resized_image = cv.resize(combined_image, dim, interpolation=cv.INTER_AREA)

        # Display labels
        labels = ["Red Isolated", "Region Masked", "Blurred + Masked", "Tracked Path"]
        positions = [(20, 30), (combined_masked.shape[1] + 20, 30), (20, combined_masked.shape[0] + 30),
                     (combined_masked.shape[1] + 20, combined_masked.shape[0] + 30)]

        for label, pos in zip(labels, positions):
            cv.putText(resized_image, label, pos, cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        if display:
            cv.imshow("Husky Tracking", resized_image)

        if cv.waitKey(1) & 0xFF == ord('q'):
            break

        frames.append(frame)

    cap.release()
    cv.destroyAllWindows()

    # Save video
    fourcc = cv.VideoWriter_fourcc(*'XVID')
    out = cv.VideoWriter(outpath + name + '.avi', fourcc, 20.0, (frames[0].shape[1], frames[0].shape[0]))
    for frame in frames:
        out.write(frame)
    out.release()

    # Save movement image
    img = frames[5].copy()
    step = 3
    colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255)]
    c_inc = 0

    for i in range(step, len(path) - step, step):
        u = path[i][0] - path[i - step][0]
        v = path[i][1] - path[i - step][1]

        if np.sqrt(u**2 + v**2) <= 50:
            cv.arrowedLine(img, (path[i - step][0], path[i - step][1]), (path[i][0], path[i][1]),
                           colors[c_inc % 3], 2, tipLength=0.4)
            c_inc += 1

    cv.imwrite(outpath + name + '.png', img)
    img = mpimg.imread(outpath + name + '.png')
    plt.imshow(img)
    plt.axis('off')
    plt.title('u, v and Orientation')
    plt.show()

    interpolate_curve(path)

## 4. Execução do Rastreamento

Agora, vamos executar a função `track_husky` para rastrear o objeto vermelho no vídeo fornecido.

In [48]:
# Caminho do vídeo de teste
test_video_path = "Video2_husky.mp4"
output_directory = "output_with_trajectory.mp4"

# Executar a função de rastreamento
track_husky(test_video_path, "tracked_husky", output_directory, display=True)

Não foi possível detectar a posição inicial.


## 5. Conclusão

Neste notebook, implementamos um sistema para rastrear a pose de um objeto vermelho em um vídeo. O sistema utiliza técnicas de processamento de imagem, como filtragem de cores, subtração de fundo e detecção de contornos, para determinar a posição e orientação do objeto. A trajetória do objeto é plotada e salva em um novo vídeo.